In [44]:
%matplotlib inline

In [45]:
from collections import defaultdict

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
import numpy as np
import wandb
import gymnasium as gym
import pandas as pd

from aux_task_discovery.utils.constants import WANDB_PROJECT, WANDB_ENTITY

In [46]:
# Set Sweep ID and agent hyperparameter to compare performance across
SWEEP_ID = "mlnlax4s"
PARAM = "learning_rate"

In [47]:
# Query W&B API for sweep runs
api = wandb.Api()
sweep = api.sweep(WANDB_ENTITY + "/" + WANDB_PROJECT + "/" + SWEEP_ID)
sweep_runs = sweep.runs

In [ ]:
max_episodes = sweep_runs[0].config['max_episodes']
n_episode_comp = round(max_episodes*0.2)
mean_episode_lens = defaultdict(list)
for run in sweep_runs:
    assert run.config['max_episodes'] == max_episodes, "Max episodes must be the same for all runs"
    run_param = run.config['agent_args'][PARAM]
    data = pd.DataFrame([row for row in run.scan_history(keys=['episode_len', 'episode'])])
    mean_episode_lens[run_param].append(data.tail(n_episode_comp)['episode_len'].mean())

n_runs = len(mean_episode_lens[run_param])
for key in mean_episode_lens:
    assert len(mean_episode_lens[key]) == n_runs, "Number of runs must be the same for all param values"
    mean_episode_lens[key] = np.mean(mean_episode_lens[key])

In [50]:
print(f"Mean episode length for last 20% of episodes averaged across {n_runs} runs")
for key, val in mean_episode_lens.items():
    print(f"{PARAM} = {key}: {val}")

Mean episode length for last 20% of episodes averaged across 2 runs
learning_rate = 0.02: 115.25
learning_rate = 0.01: 298.25
